In [1]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [2]:
input_default_json_conf_file='fcnn_emotion4_create_dataset_default.json';

In [3]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.image import load_img

2024-08-06 11:32:39.577999: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-06 11:32:41.527891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import sys
sys.path.append('../library');

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-08-06 11:32:43.613011: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-06 11:32:44.024069: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-06 11:32:44.024487: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
## Load json conf file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

# Variable globales

In [7]:
## Seed for the random variables
seed_number=0;

## Dataset 
dataset_base_dir     = DATA['dataset_base_dir'];
dataset_labels_file  = DATA['dataset_labels_file'];
dataset_name         = DATA['dataset_name'];

model_type_body = DATA['model_type_body'];
model_type_face = DATA['model_type_face'];
model_type_skel = DATA['model_type_skel'];


label_colname='label';
filename_colname='filename';
target_labels=['negative','neutro','pain','positive'];

## Output
output_base_dir = DATA['output_base_dir'];

##############################################


# If command line

In [8]:
for n in range(len(sys.argv)):
    if   sys.argv[n]=='--dataset-dir':
        dataset_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-file':
        dataset_labels_file=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-body':
        model_type_body=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-face':
        model_type_face=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-skel':
        model_type_skel=int(sys.argv[n+1]);
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];

print('   dataset_base_dir:',dataset_base_dir)
print('dataset_labels_file:',dataset_labels_file)
print('       dataset_name:',dataset_name)
print('    model_type_body:',model_type_body)
print('    model_type_face:',model_type_face)
print('    model_type_skel:',model_type_skel)
print('    output_base_dir:',output_base_dir)



   dataset_base_dir: /media/fernando/Expansion/DATASET/TESE/BER/BER2024/BER2024-BODY/
dataset_labels_file: train.csv
       dataset_name: ber2024-body
    model_type_body: efficientnet_b3
    model_type_face: efficientnet_b3
    model_type_skel: 20
    output_base_dir: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_fusion_1


# Set seed of random variables


In [9]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Loading data of dataset

In [10]:
# Load filenames and labels
train_val_df = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
print(train_val_df)

                                          filename     label
0      person7/positive/frame_count352_cam0_p0.png  positive
1          person2/pain/frame_count333_cam1_p0.png      pain
2       person8/positive/frame_count88_cam0_p0.png  positive
3      person8/negative/frame_count207_cam0_p0.png  negative
4       person7/negative/frame_count39_cam1_p0.png  negative
...                                            ...       ...
26988  person1/negative/frame_count262_cam2_p0.png  negative
26989     person7/neutro/frame_count20_cam2_p0.png    neutro
26990  person7/positive/frame_count218_cam2_p0.png  positive
26991   person5/negative/frame_count39_cam2_p0.png  negative
26992    person0/neutro/frame_count238_cam0_p0.png    neutro

[26993 rows x 2 columns]


# Creating output directory

In [11]:

output_dir = os.path.join(output_base_dir,
                          dataset_name,
                          'create_dataset_fusion',
                          model_type_body+'_'+model_type_face+'_ncod'+str(model_type_skel));

os.makedirs(output_dir,exist_ok = True);

# Create new model

In [12]:
import BodyEmotion4Lib.Classifier as bec
import FaceEmotion4Lib.Classifier as fec
import SkeletonEmotion4Lib.Classifier as sec


print('\n\nBODY');
cls_body=bec.Emotion4Classifier(model_type=model_type_body);

print('\n\nFACE');
cls_face=fec.FaceEmotion4Classifier(model_type=model_type_face);

print('\n\nSKELETON');
cls_skel=sec.Emotion4Classifier(ncod=model_type_skel);
#res=cls_body.from_img_pil(img_pil);
#res=cls_face.from_img_pil(img_pil);
#print(res);





BODY
Loading architecture efficientnet_b3

        url: https://tfhub.dev/tensorflow/efficientnet/b3/feature-vector/1
target_size: (300, 300)



2024-08-06 11:32:53.297784: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-06 11:32:53.298063: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-06 11:32:53.298266: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Loading the weights in: /home/fernando/.local/lib/python3.10/site-packages/BodyEmotion4Lib/models/model_efficientnet_b3.h5
Loaded the weights in: /home/fernando/.local/lib/python3.10/site-packages/BodyEmotion4Lib/models/model_efficientnet_b3.h5


FACE
Loaded layer with efficientnet_b3
Loading the weights in: /home/fernando/.local/lib/python3.10/site-packages/FaceEmotion4Lib/models/model_efficientnet_b3.h5
Loaded the weights in: /home/fernando/.local/lib/python3.10/site-packages/FaceEmotion4Lib/models/model_efficientnet_b3.h5


SKELETON
Loading the weights in: /home/fernando/.local/lib/python3.10/site-packages/SkeletonEmotion4Lib/models/model_onlycls_ncod20.h5
Loaded the weights in: /home/fernando/.local/lib/python3.10/site-packages/SkeletonEmotion4Lib/models/model_onlycls_ncod20.h5


# Creating PIL data images

In [13]:
from tqdm import tqdm

pil_image=[];
label_image=[];
#for index, row in train_val_df.iterrows():
for index, row in tqdm(train_val_df.iterrows(), total=len(train_val_df), desc="Processing images"):
    filepath=os.path.join(dataset_base_dir,row[filename_colname]);

    pil_img   = load_img(filepath);
    label_img = row[label_colname].lower();
    #pil_image.append(load_img(filepath,target_size=(300,300)));
    #label_image.append(target_labels.index(row[label_colname].lower()));

#print(label_image)

Processing images: 100%|██████████████████████████████████████████████████| 26993/26993 [06:54<00:00, 65.16it/s]
